In [ ]:
%pip install anthropic IPython

In [ ]:
import boto3
import json
import base64
from anthropic import AnthropicBedrock
from IPython.display import Image, JSON
from io import BytesIO

session = boto3.Session() # create a boto3 session to dynamically get and set the region name
bedrock = session.client(service_name='bedrock-runtime') #creates a Bedrock client

In [ ]:
#get a base64-encoded string from file bytes
def get_base64_from_bytes(image_bytes):
    resized_io = BytesIO(image_bytes)
    img_str = base64.b64encode(resized_io.getvalue()).decode("utf-8")
    return img_str

def transcribe_document(regular_prompt, system_prompt, image_bytes=None):
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4000,
        "temperature": 0,
        "top_k": 250,
        "top_p": 0.999,
        
        "system": system_prompt,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": get_base64_from_bytes(image_bytes),
                        },
                    },
                    {
                        "type": "text",
                        "text": regular_prompt
                    }
                ],
            }
        ],
    }
    
    return json.dumps(body)

In [ ]:
system_prompt = """
Your task is to extract all fields in <documents/> with the highest accuracy following <instructions/> 

<instructions>
  - Ensure to escape quotes in the JSON response
  - Return "" for missing field values
  - For each field in <document/>, including ALL fields in ALL <line_item/> nodes, analyze your certainty in the extracted information in <analysis/>
</instructions>

<documents type="array">
<document>
<doc_type value="Type of Document: Receipt"/>
<receipt_number value="The receipt number or other identifier number"/>
<doc_amount_total value="The total receipt amount" type="currency"/>
<date_issued value="Date document was issued" format="YYYY-MM-DDT00:00:00"/>
<vendor_business_number value="Vendor's business identification number e.g. ABN"/>
<vendor_name value="Business name issueing the receipt"/>
<vendor_address value="Vendor's site address"/>
<vendor_phone value="Vendor's phone number"/>
<payment_method value="The payment type, e.g. EFTPOS, Card"/>
<currency value="AUD/USD/CAD"/>
<all_line_items type="array">
<line_item>
<line_description value="Line item description"/>
<line_quantity value="Item quantity" default="1"/>
<line_unit_price value="Item price per unit"/>
<line_amount value="Line item $ amount" type="currency"/>
</line_item>
</all_line_items>
<all_line_items_amount_total value="Calculated sum of line item's line_amount fields"/>
</document>
</documents>

<analysis xsl="
<xsl:stylesheet>
<xsl:output method='xml'/>
<xsl:template match='all documents'>
 <xsl:element name="{name()}">
   <xsl:attribute name='value'>{[@value]}</xsl:attribute>
   <xsl:attribute name='inference'>0=EXPLICIT|1=DERIVED|2=MISSING|3=OTHER</xsl:attribute>
   <xsl:attribute name='source'>Source field only if and for explicit/extracted and derived fields</xsl:attribute>
   <xsl:apply-templates/>
 </xsl:element>
</xsl:template>
</xsl:stylesheet>
"/>
"""

regular_prompt = """
Output as JSON: { "output": "</analysis> as JSON" }
"""

image_bytes = bytearray(open("#{{receipt_file.jpeg}}", 'rb').read())

body = transcribe_document(regular_prompt, system_prompt, image_bytes)
response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")

response_body = json.loads(response.get('body').read()) 
display(response_body)
display(JSON(response_body['content'][0]['text'], root='content')) 